In [0]:
import numpy as np
import pandas as pd
import keras as ks
from keras.datasets import mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [36]:
print(x_train.shape)

(60000, 28, 28)


In [48]:
print(y_train.shape)

(60000,)


In [37]:
x_train = x_train.reshape(60000, 784)
x_test = x_test.reshape(10000, 784)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

60000 train samples
10000 test samples


In [0]:
import lightgbm as lgb

In [0]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

In [40]:
lgb.LGBMClassifier()

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.1, max_depth=-1,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
               random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
               subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [0]:
train_data = lgb.Dataset(x_train, label=y_train)
eval_data = lgb.Dataset(x_test, label=y_test, reference=train_data)

In [0]:
params = {
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'multiclass',
    'num_class': 10,
    'verbose': 2,
}

In [43]:
gbm = lgb.train(
    params,
    train_data,
    valid_sets=eval_data,
    num_boost_round=10,
    verbose_eval=5,
)

[5]	valid_0's multi_logloss: 1.26438
[10]	valid_0's multi_logloss: 0.858278


In [0]:
preds = gbm.predict(x_test)

In [0]:
y_pred = []

for x in preds:
    y_pred.append(np.argmax(x))

In [47]:
accuracy_score(y_test, y_pred)

0.9238

9割。

In [49]:
gbm2 = lgb.train(
    params,
    train_data,
    valid_sets=eval_data,
    num_boost_round=100,
    early_stopping_rounds=10,
    verbose_eval=5,
)

Training until validation scores don't improve for 10 rounds.
[5]	valid_0's multi_logloss: 1.26438
[10]	valid_0's multi_logloss: 0.858278
[15]	valid_0's multi_logloss: 0.62222
[20]	valid_0's multi_logloss: 0.474574
[25]	valid_0's multi_logloss: 0.377731
[30]	valid_0's multi_logloss: 0.309498
[35]	valid_0's multi_logloss: 0.261068
[40]	valid_0's multi_logloss: 0.225897
[45]	valid_0's multi_logloss: 0.199588
[50]	valid_0's multi_logloss: 0.178911
[55]	valid_0's multi_logloss: 0.162807
[60]	valid_0's multi_logloss: 0.149452
[65]	valid_0's multi_logloss: 0.138681
[70]	valid_0's multi_logloss: 0.129364
[75]	valid_0's multi_logloss: 0.121967
[80]	valid_0's multi_logloss: 0.115304
[85]	valid_0's multi_logloss: 0.10987
[90]	valid_0's multi_logloss: 0.10567
[95]	valid_0's multi_logloss: 0.101247
[100]	valid_0's multi_logloss: 0.0974774
Did not meet early stopping. Best iteration is:
[100]	valid_0's multi_logloss: 0.0974774


loss値下がり続けた

In [0]:
pred2 = gbm2.predict(x_test)

In [0]:
y_pred2 = []

for x in pred2:
    y_pred2.append(np.argmax(x))

In [53]:
accuracy_score(y_test, y_pred2)

0.9727

97%。まだいけそうか

In [54]:
gbm3 = lgb.train(
    params,
    train_data,
    valid_sets=eval_data,
    num_boost_round=500,
    early_stopping_rounds=100,
    verbose_eval=100,
)

Training until validation scores don't improve for 100 rounds.
[100]	valid_0's multi_logloss: 0.0974774
[200]	valid_0's multi_logloss: 0.0680255
[300]	valid_0's multi_logloss: 0.0632429
[400]	valid_0's multi_logloss: 0.0648402
Early stopping, best iteration is:
[301]	valid_0's multi_logloss: 0.0631857


In [0]:
pred3 = gbm3.predict(x_test)

In [0]:
y_pred3 = []

for x in pred3:
    y_pred3.append(np.argmax(x))

In [57]:
accuracy_score(y_test, y_pred3)

0.9806

98%。良好か。